In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam


In [ ]:
from google.colab import files
files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"parv1509","key":"131cec0ea91b12befcd5e48e87ec7460"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!kaggle datasets download -d biaiscience/dogs-vs-cats


Dataset URL: https://www.kaggle.com/datasets/biaiscience/dogs-vs-cats
License(s): DbCL-1.0
 99% 809M/817M [00:03<00:00, 245MB/s]
100% 817M/817M [00:03<00:00, 239MB/s]


In [ ]:
import zipfile

with zipfile.ZipFile("dogs-vs-cats.zip", "r") as zip_ref:
    zip_ref.extractall("data")


In [ ]:
import os
os.listdir("data")


['test', 'train']

In [ ]:
import os, shutil, random

base_dir = "dataset"
train_dir = base_dir + "/train"
val_dir = base_dir + "/val"

os.makedirs(train_dir + "/cats", exist_ok=True)
os.makedirs(train_dir + "/dogs", exist_ok=True)
os.makedirs(val_dir + "/cats", exist_ok=True)
os.makedirs(val_dir + "/dogs", exist_ok=True)


In [ ]:
images = os.listdir("data/train/train")
random.shuffle(images)

split = int(0.8 * len(images))
train_images = images[:split]
val_images = images[split:]

def move_images(image_list, target_dir):
    for img in image_list:
        label = "cats" if img.startswith("cat") else "dogs"
        shutil.copy(
            "data/train/train/" + img,
            target_dir + "/" + label + "/" + img
        )

move_images(train_images, train_dir)
move_images(val_images, val_dir)

In [ ]:
images = os.listdir("data/train/train")
random.shuffle(images)

split = int(0.8 * len(images))
train_images = images[:split]
val_images = images[split:]

def move_images(image_list, target_dir):
    for img in image_list:
        label = "cats" if img.startswith("cat") else "dogs"
        shutil.copy(
            "data/train/train/" + img,
            target_dir + "/" + label + "/" + img
        )

move_images(train_images, train_dir)
move_images(val_images, val_dir)

In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

base_model = MobileNetV2(
    weights="imagenet",
    include_top=False,
    input_shape=(224,224,3)
)

base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation="relu")(x)
output = Dense(1, activation="sigmoid")(x)

model = Model(base_model.input, output)


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

val_datagen = ImageDataGenerator(rescale=1./255)

train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

val_data = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=10
)

Found 23968 images belonging to 2 classes.
Found 8968 images belonging to 2 classes.


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
749/749 ━━━━━━━━━━━━━━━━━━━━ 347s 440ms/step - accuracy: 0.9388 - loss: 0.1452 - val_accuracy: 0.9854 - val_loss: 0.0417
Epoch 2/10
749/749 ━━━━━━━━━━━━━━━━━━━━ 303s 405ms/step - accuracy: 0.9615 - loss: 0.0962 - val_accuracy: 0.9749 - val_loss: 0.0654
Epoch 3/10
749/749 ━━━━━━━━━━━━━━━━━━━━ 308s 412ms/step - accuracy: 0.9655 - loss: 0.0836 - val_accuracy: 0.9711 - val_loss: 0.0689
Epoch 4/10
749/749 ━━━━━━━━━━━━━━━━━━━━ 313s 417ms/step - accuracy: 0.9703 - loss: 0.0738 - val_accuracy: 0.9875 - val_loss: 0.0332
Epoch 5/10
749/749 ━━━━━━━━━━━━━━━━━━━━ 324s 433ms/step - accuracy: 0.9718 - loss: 0.0733 - val_accuracy: 0.9865 - val_loss: 0.0342
Epoch 6/10
749/749 ━━━━━━━━━━━━━━━━━━━━ 306s 408ms/step - accuracy: 0.9709 - loss: 0.0720 - val_accuracy: 0.9845 - val_loss: 0.0385
Epoch 7/10
749/749 ━━━━━━━━━━━━━━━━━━━━ 307s 410ms/step - accuracy: 0.9715 - loss: 0.0695 - val_accuracy: 0.9868 - val_loss: 0.0357
Epoch 8/10
749/749 ━━━━━━━━━━━━━━━━━━━━ 308s 412ms/step - accuracy: 0.9717 -

In [ ]:
model.save("cats_vs_dogs_transfer_learning.h5")


#  Day 18 – Project 2


In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense


In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train / 255.0
x_test = x_test / 255.0

x_train = x_train.reshape(-1,28,28,1)
x_test = x_test.reshape(-1,28,28,1)


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
model = Sequential([
    Conv2D(32, (3,3), activation="relu", input_shape=(28,28,1)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation="relu"),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(128, activation="relu"),
    Dense(10, activation="softmax")
])


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

model.fit(x_train, y_train, epochs=5, validation_data=(x_test, y_test))


Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.9143 - loss: 0.2820 - val_accuracy: 0.9861 - val_loss: 0.0400
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9865 - loss: 0.0415 - val_accuracy: 0.9893 - val_loss: 0.0334
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9914 - loss: 0.0276 - val_accuracy: 0.9897 - val_loss: 0.0269
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - accuracy: 0.9942 - loss: 0.0179 - val_accuracy: 0.9913 - val_loss: 0.0275
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9956 - loss: 0.0139 - val_accuracy: 0.9897 - val_loss: 0.0347


In [ ]:
model.save("mnist_digit_model.h5")
